In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import getcwd
from scipy.constants import e, k

inp_dir = getcwd() + '/input_files/'

temp = 25
T = temp + 273
v_t = k * T / e

In [ ]:
def read_file(filename, sep=',', outp_type=float):
    with open(filename) as file:
        lines = file.readlines()
        res = []
        for line in lines:
            res += [outp_type(i) for i in line.split(sep) if line]
            
    return res


def write_file(filename, data, sep=' '):
    with open(filename, 'w') as file:
        for line in data:
            str_line = str(line) + '\n' # sep.join(line) + '\n'
            file.write(str_line)


def rmse(arr1, arr2):
    diff = (arr1 - arr2) ** 2
    mse = np.sum(diff) / arr1.size
    rmse = np.sqrt(mse)
    return rmse


def minimize(init_cur, eta_min=0.5, eta_max=3, steps=500):
    min_error = 10 ** 10
    
    for eta in np.linspace(eta_min, eta_max, steps):
        approx = init_cur * np.exp(v / v_t / eta - 1)
        error = rmse(i, approx)
        errors.append(error)
    
        if error < min_error:
            min_error = error
            min_eta = eta
        
    return min_eta, min_error


def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1,
                       length=100, fill='█', print_end='\r'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=print_end)
    # Print New Line on Complete
    if iteration == total:
        print()

In [ ]:
data = read_file(inp_dir + 'diode_iv_curve.txt')
v = np.array([val for i, val in enumerate(data) if not i % 2])
i = np.array([val for i, val in enumerate(data) if i % 2])

write_file(inp_dir + 'diode_i_curve.txt', i)
write_file(inp_dir + 'diode_u_curve.txt', v)

In [ ]:
zero_cur = init_cur = i[0] / np.exp(v[0] / v_t - 1)
target_error = 0.1


while True:
    eta, error = minimize(init_cur)
    init_cur = i[0] / (np.exp(v[0] / v_t / eta) - 1)
    metric = abs(init_cur - zero_cur) / zero_cur * 100
    print(init_cur, zero_cur, metric)
    
    
    if metric < target_error:
        break
        
print(init_cur, eta)

In [ ]:
approx = init_cur * (np.exp(v / v_t / eta) - 1)
approx1 = init_cur * (np.exp(v / v_t / eta) - 1)

plt.plot(v, approx)
plt.plot(v, i)

In [ ]:
steps = 100
min_error = 10 ** 10
x_fit_vals = v[-12:]
y_fit_vals = i[-12:]

init_cur = i[0] / (np.exp(v[0] / v_t) - 1)
fit = np.polyfit(x_fit_vals, y_fit_vals, 1)
init_rs = 1 / fit[0]


for idx, eta in enumerate(np.linspace(1, 2, steps)):
    print_progress_bar(idx + 1, steps)
    for cur in np.linspace(0.75 * init_cur, 1.25 * init_cur, steps):
        for rs in np.linspace(0.75 * init_rs, 1.25 * init_rs, steps):
            approx = cur * (np.exp((v - rs * i) / v_t / eta) - 1)
            error = rmse(i, approx)
            errors.append(error)
    
            if error < min_error:
                min_error = error
                min_eta, min_cur, min_rs = eta, cur, rs
                
print(min_eta, min_cur, min_rs)

In [ ]:
eps = 10e-2
eta = 0.1

a1 = min_cur * (np.exp((v - min_rs * i) / v_t / min_eta) - 1)

ii = [0]
vv = np.linspace(v[0], v[-1], 100)

for idx, v_val in enumerate(vv[1:]):    
    old_c = ii[-1]
    new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
    
    while abs(new_c - old_c) > eps:
        old_c = new_c
        new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
        new_c = old_c + eta * (new_c - old_c)
    
    ii.append(new_c)


# plt.plot(v, a1)
plt.plot(vv, ii)
plt.plot(v, i)
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('I-V Curve of Diode')
plt.grid(True)
plt.show()